In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
# df is your output dataframe with columns:
# 'trade_date_year_mo', 'delivery_year_mo', 'weighted_avg_price', 'total_transacted_quantity'
# use 2019 as test
df = pd.read_csv(r"/Users/ankitakhatri/Downloads/STA 160/EQR_master_output_new.csv")

In [3]:
# re-convert them to datetimes for sorting:
df['trade_date_year_mo'] = pd.to_datetime(df['trade_date_year_mo'])
df['delivery_year_mo']   = pd.to_datetime(df['delivery_year_mo'])

# output folder
out_dir = "Price Graphs Pre-EDA"
os.makedirs(out_dir, exist_ok=True)

# loop over each delivery month and make a chart
for delivery_month, g in df.groupby('delivery_year_mo'):
    # sort by trade date so the x-axis is ordered
    g = g.sort_values('trade_date_year_mo')
    
    # expand trade_date so there are no missing months → no line breaks
    full_range = pd.date_range(
        start=g['trade_date_year_mo'].min(),
        end=g['trade_date_year_mo'].max(),
        freq='MS'           # Month Start frequency
    )

    g = g.set_index('trade_date_year_mo').reindex(full_range)
    g.index.name = 'trade_date_year_mo'

    # interpolate missing values so the line stays connected
    g['weighted_avg_price'] = g['weighted_avg_price'].interpolate()
    g['total_transacted_quantity'] = g['total_transacted_quantity'].fillna(0)

    fig, ax1 = plt.subplots(figsize=(10, 6))

    # left y-axis: weighted average price (connected line, no dots)
    ax1.plot(
        g.index,                     # <-- use index, not column
        g['weighted_avg_price'],
        linewidth=2,
        linestyle='-'
    )
    ax1.set_xlabel("Trade Date (YYYY-MM)")
    ax1.set_ylabel("Weighted Avg Price ($/KW-MO)")

    # format x-axis as YYYY-MM
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    # right y-axis: total transacted quantity (wider, darker bars)
    ax2 = ax1.twinx()
    ax2.bar(
        g.index,                     # <-- use index here too
        g['total_transacted_quantity'],
        width=20,
        alpha=0.4,
        color='black'
    )
    ax2.set_ylabel("Total Transacted Quantity (MW)")

    # title + layout
    month_str = delivery_month.strftime('%Y-%m')
    ax1.set_title(f"Delivery Month: {month_str}")
    fig.autofmt_xdate(rotation=45)
    plt.tight_layout()

    # save figure as image
    save_path = os.path.join(out_dir, f"{month_str}.jpeg")
    fig.savefig(save_path, dpi=300)
    plt.close(fig)
